In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

Load the saved model, encoders, and scaler

In [5]:
model = load_model('model.h5')
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('one_hot_encoder.pkl', 'rb') as f:
    one_hot_encoder = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [6]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender' : 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [7]:
geo_encoded = one_hot_encoder.transform([[input_data['Geography']]])
geo_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_df

C:\Users\ronsi\PycharmProjects\ANN-Classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [9]:
input_data = pd.concat([input_df.drop('Geography', axis=1), geo_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


Encode the categorical variables

In [10]:
input_data['Gender'] = label_encoder_gender.transform([input_data['Gender']])
input_data

C:\Users\ronsi\PycharmProjects\ANN-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


Scale the features

In [11]:
input_data_scaled = scaler.transform(input_data)
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

Predict the outcome

In [12]:
prediction = model.predict(input_data_scaled)
predicted_class = (prediction > 0.5).astype(int)
print(f'Predicted class: {predicted_class[0][0]}')
print(f'Prediction probability: {prediction[0][0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Predicted class: 0
Prediction probability: 0.0402611643075943
